<a href="https://colab.research.google.com/github/Szymqn/ProgramowanieRownolegleIRozproszone/blob/master/lab4/zadanie1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Zaimplementuj wariant szeregowy oraz równoległy następujących algorytmów

a. całkowanie numeryczne wybranej funkcji f(x) w przedziale <a,b> metodą trapezów

https://pl.wikipedia.org/wiki/Ca%C5%82kowanie_numeryczne

b. wyznaczanie pola koła metodą Monte Carlo

https://pl.wikipedia.org/wiki/Metoda_Monte_Carlo

c. generowanie fraktali Mandelbrota oraz Julii

https://pl.wikipedia.org/wiki/Zbi%C3%B3r_Mandelbrota

https://pl.wikipedia.org/wiki/Zbi%C3%B3r_Julii

d. wybrany inny algorytm numeryczny, który da sie zrównoleglić